# Principle Component Analysis and Clustering of Hospital Quarterly Financial Data
### Jeevitha Bandi, Doah Karabey, Elling Payne

## 2.4 Technical Background - Agglomerative Hierarchical Clustering

In hierarchical clustering (AHC) the goal is build a hierarchical map of relationships between clusters. In agglomerative clustering, this is done iteratively and starting from the lowest level, which is relationships between individual data points. At each iteration, the two most similar clusters are linked, and this link informs the next iteration that the two clusters are now one. This process continues until all clusters have been linked together, with each iteration presenting a clustering with fewer and fewer clusters. Which two clusters are most similar at a given step depends mainly on the _linkage type_ and the _distance measure_ used. The most common _distance measure_ is the Euclidean distance between two points in feature space. The _linkage type_ refers to which points in each cluster matter for calculating cluster distance, as well as how they are aggregated into a single distance metric for every cluster pair. Three of the most common are _single linkage_, _complete linkage_, and _average linkage_.

When compared with K-means clustering, the process of hierarchical clustering produces multiple clustering solutions with the cluster counts ranging from one to the size of the data set, rather than requiring that the number of clusters be set beforehand. As a bonus, the process of building the hierarchy can be represented nicely in a _dendogram_. The height of a dendogram corresponds to the similarity between the most sinmilar clusters. A choice of clustering step corresponds to cutting the dendogram at a particular height. Another way of looking at it is that cutting the dendogram at a particular height is in fact a selection of the minimum cluster dissimilarity to consider two clusters truly different. Then only those cluster splits that results in clusters at least that dissimilar will be retained.

### 2.4.1 Single Linkage
When using single linkage, the linkage distance between two clusters only depends on the closest pair of points in the two clusters. This might be useful in a scenario in which the closest points in a cluster are the most important for understanding the relationships of interest. For instance, one might be able to imagine a social process in which two groups are more likely to come into contact if they each have one member that can kick things off with the someone in the other group. If all we were interested in was attempting to group friend groups, it is concievable that given two groups, the most similar members matter more than members that are dissimilar. However, in practice this is fairly uncommon, and single linkage tends to result in pretty unbalanced clusters. This is because the largest two clusters at any given step are the most likely to be the closest.


### 2.4.2 Complete Linkage
In complete linkage, the distance between two clusters is based on the pair of points which are furthest from eachother. This might be considered the most conservative of the three linkage types mentioned in that it tends to enforce a more balanced clustering. In contrast with single linkage, the largest two clusters are now the least likely to be the most similar. However, in some scenarios this approach might result in single outliers being merged into a cluster before another cluster which by other measures might be considered more similar.

### 2.4.3 Average Linkage
Average linkage considers the average of all of the pairwise similarity scores between points in the two clusters. This is not to be confused with _centroid linkage_, which is not used in this project. Average linkage is a balanced approach compared with single and complete linkage. Like complete linkage, it tends to create more balanced clusters than single linkage, but to a lesser degree.

## 4.4 Results - Agglomerative Hierarchical Clustering

In general, with our current data preprocessing procedures, hierarchical clustering does not yield incredibly useful results. Of the three linkage types used, only complete linkage produced multiple clusters with more than one financial record. All three linkage types resulted in extremely unbalanced clusters on the data set, with all silhouette scores being essentially zero. As such, only plots of within group sum of squares and cluster scatter plots are included to compare models with different numbers of clusters.

### 4.4.1 Single Linkage

![Single Linkage Clusters by K](data/hierarchical_clustering/single_linkage_clusters.png)

![Single Linkage WSS by K](data/hierarchical_clustering/single_linkage_wss_vs_k.png)

![Single Linkage Subset Dendogram](data/hierarchical_clustering/single_linkage_dendogram.png)

### 4.4.2 Complete Linkage

The best hierarchical model seemed to used complete linkage and 4 clusters, which results in most of the data points in one group with a small offshoot group that includes 10 financial records. This smaller group of records was mostly from Yakima Valley Memorial, with one record from Confluence Health: Central Washington Hospital. This group was mostly distinguished by a higher than average focus on swing beds, and also by a higher than average number of patients paying out of pocket or with commercial insurance. The model with 6 clusters is also relatively interesting. It identifies the same small cluster in the 4 cluster model and identifies one additional small group of size 8. This additional cluster includes records from Snoqualmie Valley Hospital, which appears to be larger than average by most metrics. However, both of the small groups are composed of multiple records from only one ore two hospitals. As such they still really represent outliers rather than useful groups. The following table shows selected values from the normalized centroids of clusters larger than 1 in the 6-cluster model. Cluster 0 is the main cluster, while clusters 1 and 2 are the smaller clusters. Looking at the plot of within group sum of squares, there is a slight elbow at 4 and 6 clusters, which might lend some weak support to these clusterings. However, the silhouette scores contradict this and reduce confidence in the clustering.

| Cluster | Licensed Beds | Total Discharges | Swing Bed Total Revenue | Total Self-Pay Discharges | Total Revenue | Total Operating Expenses |
|:--:|:--:|:--:|:--:|:--:|:--:|:--:|
| 0 | 0.001999981 | -0.001821415 | -0.044833685 | -0.019696759 | 4.11E-05 | -0.001246769 |
| 1 | -0.699417057 | 0.071811687 | 11.59348497 | 3.930027732 | -0.665500625 | -0.568140766 |
| 2 | 0.434480284 | 0.803617515 | -0.218571816 | 2.124113904 | 0.851412059 | 0.796355729 |

![Complete Linkage Clusters by K](data/hierarchical_clustering/complete_linkage_clusters.png)

![Complete Linkage WSS by K](data/hierarchical_clustering/complete_linkage_wss_vs_k.png)

![Complete Linkage Subset Dendogram](data/hierarchical_clustering/complete_linkage_dendogram.png)

### 4.4.3 Average Linkage

![Average Linkage Clusters by K](data/hierarchical_clustering/avg_linkage_clusters.png)

![Average Linkage WSS by K](data/hierarchical_clustering/avg_linkage_wss_vs_k.png)

![Average Linkage Subset Dendogram](data/hierarchical_clustering/avg_linkage_dendogram.png)

## 5.1 Discussion - Singular Values and Scree Plots

## 5.4 Discussion - Clustering Interpretations

The hope is that the clusterings here will represent meaningful clusters where the clustering unit is not hospitals but individual financial quarters at various hospitals. As such, some expected outcomes are that different quarters from the same hospital are likely to be in the same cluster, unless some drastic change occured at that hospital. This is visible in the dendogram for agglomerative clustering with complete linkage, in which quarterly records from teh same hospital typically occur in the same groups.

## 7 References